In [68]:
from collections import Counter
import math
import requests
import json

In [48]:
corpus_of_documents = [
    "The capital of France is Paris.",
    "Delhi is capital of india",
    "The Great Barrier Reef is the world's largest coral reef system.",
    "Retrieval-Augmented Generation helps LLMs use private data.",
    "India gate is in delhi"
]



In [4]:
user_query = "i am an indian and i live in india"

In [22]:
document = "india is a country for the indians and for everyone"

In [69]:
query_tokens = user_query.lower().split(" ")
query_tokens

['i', 'am', 'an', 'indian', 'and', 'i', 'live', 'in', 'india']

In [24]:
document_tokens = document.lower().split(" ")
document_tokens

['india',
 'is',
 'a',
 'country',
 'for',
 'the',
 'indians',
 'and',
 'for',
 'everyone']

In [25]:
query_counter = Counter(query_tokens)

In [26]:
document_counter = Counter(document_tokens)

In [27]:
lst = []
for token in query_counter.keys():
    lst.append(query_counter[token])

In [28]:
lst

[2, 1, 1, 1, 1, 1, 1, 1]

In [29]:
mylist = []
for tokens in query_counter.keys() & document_counter.keys():
    mylist.append(query_counter[tokens] * document_counter[tokens])  #dot product(word level similarity)

In [30]:
mylist

[1, 1]

In [39]:
dot_product = sum(mylist)

In [40]:
query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))
query_magnitude

3.3166247903554

In [41]:
document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))
document_magnitude

3.4641016151377544

In [42]:
similarity = (dot_product)/(query_magnitude*document_magnitude)
similarity

0.17407765595569785

In [43]:
def cosine_similarity(query, document):
    query_tokens = query.lower().split(" ")
    document_tokens = document.lower().split(" ")

    #creating counters () vectors
    query_counter = Counter(query_tokens)
    document_counter = Counter(document_tokens)

    #dot product
    dot_product = sum(query_counter[token] * document_counter[token] for token in query_counter.keys())

    #calulating the manitudes
    query_magnitude = math.sqrt(sum(query_counter[token]**2 for token in query_counter))
    document_magnitude = math.sqrt(sum(document_counter[token]**2 for token in document_counter))

    #calculating the cosine similarity
    similarity = dot_product/(query_magnitude * document_magnitude)
    return similarity

In [45]:
cosine_similarity(user_query, document)

0.17407765595569785

In [46]:
def return_responses (query, corpus):
    similarities = []
    for doc in corpus:
        similarity = cosine_similarity(query, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [49]:
return_responses(user_query, corpus_of_documents)

'India gate is in delhi'

In [52]:
user_input = user_query

In [55]:
relevant_document=return_responses(user_input,corpus_of_documents)

In [56]:
relevant_document

'India gate is in delhi'

In [66]:
#using llama 2 i am going to do augmentation
full_response = []
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = 'http://localhost:11434/api/generate'


data = {
    "model": "llama2",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)


try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            # print(decoded_line['response'])  # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()
    
    
print(''.join(full_response))

 "Explore the Qutub Minar, a UNESCO World Heritage Site located near India Gate in Delhi."
